In [1]:
import jax
import jax.numpy as jnp
import inference.mlp as mlp

In [2]:
c = 1
h = 60
w = 80
sizes = [512, 256, 128]

key = jax.random.PRNGKey(42)

weights = mlp.qs_mlp(c, h, w, sizes, key, 1)

Platform 'METAL' is experimental and not all JAX functionality may be correctly supported!
2024-05-18 21:43:36.048415: W pjrt_plugin/src/mps_client.cc:563] WARNING: JAX Apple GPU support is experimental and not all JAX functionality is correctly supported!


Metal device set to: Apple M2

systemMemory: 24.00 GB
maxCacheSize: 8.00 GB



In [7]:
import pickle

# # Specify the file path
# file_path = "weights/weights_test.pkl"

# # Save the weights to the file
# with open(file_path, "wb") as file:
#     pickle.dump(weights, file)

# # Load the weights from the file
# with open(file_path, "rb") as file:
#     loaded_weights = pickle.load(file)

# # Check if the loaded weights are the same as the original weights
# for i in range(len(weights)):
#     for j in range(len(weights[i])):
#         assert jnp.allclose(weights[i][j], loaded_weights[i][j])

In [8]:
# import os
# mlp.mlp_serialize_binary(weights, 'weights/weights_test.bin')
# print(os.path.getsize('weights/weights_test.bin'))


20320296


In [9]:
with open('weights/weights_test.pkl', 'rb') as file:
    loaded_weights = pickle.load(file)
    
x = jnp.ones((2, c, h, w), dtype=jnp.float32).flatten()
output = mlp.mlp_forward(loaded_weights, x)


SHAPES (9600,) (9600, 512) (512,)
SHAPES (512,) (512, 256) (256,)
SHAPES (256,) (256, 128) (128,)
SHAPES (128,) (128, 1) (1,)


In [6]:
output

Array([0.70070374], dtype=float32)